# Aplicación de filtro de partículas a seguimiento visual

Actividad realizada por Sergio Hernández García para el curso de Introducción a los Métodos Bayesianos para la Inferencia Estadística 2022.

En este notebook se realiza un ejemplo básico de aplicación de los contenido vistos sobre filtro de partículas a un caso de seguimiento visual de una pelota.

In [34]:
import time
import cv2
import os
import numpy as np
import random

In [2]:
def calcular_pesos(img, particulas):
    h, w = 30, 30
    estados = particulas['estados']

    for i in range(estados.shape[0]):
        y = estados[i, 0]
        x = estados[i, 1]
        particulas['peso'][i] = np.sum(img[y:y+h, x:x+w])

    total = np.sum(particulas['peso'])

    if total != 0:
        particulas['peso'] = particulas['peso']/total

In [3]:
def select_best_particle(particulas):
    index = np.argmax(particulas['peso'])
    if particulas['peso'][index] > 0:
        return particulas['estados'][index,0], particulas['estados'][index,1]
    else:
        return None, None

In [4]:
def select_particles(particulas):

    y_best, x_best = select_best_particle(particulas)

    pesos_acum = particulas['peso']

    for i in range(1, pesos_acum.shape[0]):
        pesos_acum[i] = pesos_acum[i-1] + pesos_acum[i]

    random.seed(None)

    for i in range(pesos_acum.shape[0]):
        rand = random.random()
        j = 0

        while j < pesos_acum.shape[0] and pesos_acum[j] < rand:
            j = j+1

        if j < pesos_acum.shape[0]:
            mu, sigma = 0, 20  # mean and standard deviation

            particulas['estados'][i,0] = particulas['estados'][j,0] + int(np.random.normal(mu, sigma))
            particulas['estados'][i,1] = particulas['estados'][j,1] + int(np.random.normal(mu, sigma))

    return y_best, x_best

In [49]:
def print_particles(img, particulas, y_best = None, x_best = None):
    h, w = 30, 30

    # img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    if particulas:
        index = np.random.choice(range(len(particulas['estados'])), int(len(particulas['estados'])*0.5), replace=False)
        estados = particulas['estados'][index]
        peso_p = particulas['peso'] + 1
        print(np.sum(peso_p))
        for i in range(estados.shape[0]):
            y = estados[i, 0]
            x = estados[i, 1]
            cv2.circle(img, center=(int(x+w/2), int(y+h/2)), radius=int(peso_p[i] * 3), color=(0, 255, 0), thickness=-1)
            # cv2.rectangle(img, (x,y),(x+w,y+h), (0, 255, 0), 1)

    if y_best and x_best:
        cv2.rectangle(img, (int(x_best-w/2), int(y_best-h/2)), (int(x_best + w/2), int(y_best + h/2)), (0, 0, 255), 2)

    cv2.imshow('frame', img)
    cv2.waitKey(1)

In [36]:
ruta = "SecuenciaPelota"
ruta_img = [os.path.join(ruta, '{}.jpg'.format(i)) for i in range(1, 61)]

img = cv2.imread(ruta_img[0], cv2.IMREAD_GRAYSCALE)

n = 200
particulas = {'estado': np.zeros((n,2)),
              'peso': np.zeros(n)}

y_best, x_best = None, None

Con fines demostrativos, leemos el fotograma número 10.

In [37]:
frame_rgb = cv2.imread(ruta_img[10], cv2.IMREAD_COLOR)
frame = cv2.cvtColor(frame_rgb, cv2.COLOR_BGR2GRAY)

In [38]:
cv2.imshow('frame', frame_rgb)
cv2.waitKey(1)

-1

## Mueatrear n partículas
En los primeros pasos muestreamos de una distribución uniforme para cubrir toda la imagen. Una ved se ha detectado el objeto a seguir, pasaremos a muestrear de una distribución Normal.

In [45]:
particulas['estados'] = np.random.rand(n, 2) * frame.shape[0]
particulas['estados'] = particulas['estados'].astype(int)

In [50]:
print_particles(frame_rgb, particulas)

error: OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - Argument 'radius' is required to be an integer
>  - Argument 'radius' is required to be an integer


Para detectar la pelota, realizamos una sustracción de fondo.

In [41]:
# Sustraccion de fondo
th = np.abs(np.float32(frame) - np.float32(img)) > 70
th = np.uint8(th * 255)

In [42]:
cv2.imshow('umbralizado', th)
cv2.waitKey(1)

QObject::moveToThread: Current thread (0x560c2ea1eb30) is not the object's thread (0x560c2eb8dc40).
Cannot move to target thread (0x560c2ea1eb30)

QObject::moveToThread: Current thread (0x560c2ea1eb30) is not the object's thread (0x560c2eb8dc40).
Cannot move to target thread (0x560c2ea1eb30)

QObject::moveToThread: Current thread (0x560c2ea1eb30) is not the object's thread (0x560c2eb8dc40).
Cannot move to target thread (0x560c2ea1eb30)

QObject::moveToThread: Current thread (0x560c2ea1eb30) is not the object's thread (0x560c2eb8dc40).
Cannot move to target thread (0x560c2ea1eb30)

QObject::moveToThread: Current thread (0x560c2ea1eb30) is not the object's thread (0x560c2eb8dc40).
Cannot move to target thread (0x560c2ea1eb30)

QObject::moveToThread: Current thread (0x560c2ea1eb30) is not the object's thread (0x560c2eb8dc40).
Cannot move to target thread (0x560c2ea1eb30)

QObject::moveToThread: Current thread (0x560c2ea1eb30) is not the object's thread (0x560c2eb8dc40).
Cannot move to tar

-1

Calculamos la likelihood de cada partícula. Para esto, realizamos un recuento del número de píxeles pertenecientes a la pelota que se encuentran dentro de un rectángulo de tamaño 30x30 con centro en las coordenadas de cada partícula.

In [43]:
h, w = 30, 30
estados = particulas['estados']

for i in range(estados.shape[0]):
    y = estados[i, 0]
    x = estados[i, 1]
    particulas['peso'][i] = np.sum(img[int(y-h/2):int(y+h/2), int(x-w/2):int(x+w/2)])

# Normalizamos los pesos
total = np.sum(particulas['peso'])
particulas['peso'] = particulas['peso']/total

Remuestreamos de la likelihood para seleccionar una partícula.

In [24]:
# index = np.argmax(particulas['peso'])
index = np.random.choice(range(n), p=particulas['peso'])

y_best = particulas['estados'][index,0]
x_best = particulas['estados'][index,1]

print('x=', x_best, ' y=', y_best, ' peso=', particulas['peso'][index])

[0.00530476 0.00533508 0.00450461 0.         0.00543316 0.00529315
 0.00460791 0.00521002 0.005172   0.00528575 0.00483874 0.00532289
 0.         0.00527937 0.00486457 0.00524992 0.00534321 0.00524513
 0.00532086 0.0053168  0.00490389 0.00532463 0.00476054 0.00507799
 0.00508118 0.00449025 0.00520929 0.00489736 0.00521611 0.00530113
 0.00534973 0.005302   0.00483396 0.         0.00512355 0.00520741
 0.00543766 0.0052695  0.00526704 0.00439057 0.005237   0.0050378
 0.00510889 0.00529431 0.00485557 0.00525499 0.00500951 0.00527095
 0.00453856 0.00515488 0.00527298 0.00521597 0.00514212 0.00530374
 0.00464375 0.         0.00442728 0.00511455 0.00532289 0.00527327
 0.00530374 0.00534422 0.00529272 0.00525296 0.00516519 0.0049567
 0.00504462 0.0052695  0.00506116 0.00484005 0.00440262 0.00504346
 0.00489112 0.0053287  0.00520218 0.00497846 0.03318626 0.00500371
 0.00533682 0.00474836 0.00457135 0.00522264 0.00529025 0.0051987
 0.         0.00447545 0.00531709 0.00510933 0.00528575 0.0045632

In [25]:
print_particles(frame_rgb, particulas, y_best, x_best)

# cv2.waitKey()

In [ ]:
pesos_acum = particulas['peso']

for i in range(1, pesos_acum.shape[0]):
    pesos_acum[i] = pesos_acum[i-1] + pesos_acum[i]

random.seed(None)

for i in range(pesos_acum.shape[0]):
    rand = random.random()
    j = 0

    while j < pesos_acum.shape[0] and pesos_acum[j] < rand:
        j = j+1

    if j < pesos_acum.shape[0]:
        mu, sigma = 0, 20  # mean and standard deviation

        particulas['estados'][i,0] = particulas['estados'][j,0] + int(np.random.normal(mu, sigma))
        particulas['estados'][i,1] = particulas['estados'][j,1] + int(np.random.normal(mu, sigma))

In [8]:
for img in ruta_img:
    frame_rgb = cv2.imread(img, cv2.IMREAD_COLOR)
    frame = cv2.cvtColor(frame_rgb, cv2.COLOR_BGR2GRAY)

    if not y_best and not x_best:
        particulas['estados'][:, 0] = np.random.rand(n) * frame.shape[0]
        particulas['estados'][:, 1] = np.random.rand(n) * frame.shape[1]
        particulas['estados'] = particulas['estados'].astype(int)

    # Sustraccion de fondo
    th = np.abs(np.float32(frame) - np.float32(img)) > 70
    th = np.uint8(th * 255)

    #calculo pesos
    calcular_pesos(th, particulas)

    # print_particles(frame_rgb, particulas)

    y_best, x_best = select_particles(particulas)

    print_particles(frame_rgb, particulas, y_best, x_best)

    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

    time.sleep(0.2)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x560c2ea1eb30) is not the object's thread (0x560c2eb8dc40).
Cannot move to target thread (0x560c2ea1eb30)

QObject::moveToThread: Current thread (0x560c2ea1eb30) is not the object's thread (0x560c2eb8dc40).
Cannot move to target thread (0x560c2ea1eb30)

QObject::moveToThread: Current thread (0x560c2ea1eb30) is not the object's thread (0x560c2eb8dc40).
Cannot move to target thread (0x560c2ea1eb30)

QObject::moveToThread: Current thread (0x560c2ea1eb30) is not the object's thread (0x560c2eb8dc40).
Cannot move to target thread (0x560c2ea1eb30)

QObject::moveToThread: Current thread (0x560c2ea1eb30) is not the object's thread (0x560c2eb8dc40).
Cannot move to target thread (0x560c2ea1eb30)

QObject::moveToThread: Current thread (0x560c2ea1eb30) is not the object's thread (0x560c2eb8dc40).
Cannot move to target thread (0x560c2ea1eb30)

QObject::moveToThread: Current thread (0x560c2ea1eb30) is not the object's thread (0x560c2eb8dc40).
Cannot move to tar